In [1]:
import os
import shutil
from shutil import copyfile
import sys
import cv2
import json
import numpy as np

import os
import sys
import cv2
import json
import numpy as np



def resize_focus_window(fullscreen=False):
    # Set the desired display window size
    frame_width = int(1920 / 2)
    frame_height = int(1080 / 2)
    # Create a resizable display window
    cv2.namedWindow('Video', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Video', frame_width, frame_height)
    # Focus window
    cv2.setWindowProperty('Video', cv2.WND_PROP_TOPMOST, 1)
    if fullscreen:
        # Set window to full screen mode
        cv2.setWindowProperty('Video', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        

def visualize_video(path_to_raw_video, path_to_folder_with_json, slowdown=0):
    # Open the video file
    video_name= os.path.split(path_to_raw_video)[1]
    cap = cv2.VideoCapture(path_to_raw_video)
    # Check if video file opened successfully
    if not cap.isOpened():
        sys.exit("Error opening video file")

        # Totla number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    cv2.startWindowThread()
    resize_focus_window()

    json_files = os.listdir(path_to_folder_with_json)
    json_files.sort()
    track_id_dict = {}
    playing = True
    frame_counter = 0
    # Loop through each frame in the video
    while True:
        if playing:
            # Read a frame from the video
            ret, frame = cap.read()
            # If frame read successfully, add annotations and display it
            if ret:
                # Load frame data
                json_file = json_files[frame_counter]
                with open(os.path.join(path_to_folder_with_json, json_file), 'r') as f:
                    data = json.load(f)

                # Add bbox and ID
                for person in data:
                    bbox = person['bbox']
                    track_id = person['track_id']
                    if track_id not in track_id_dict:
                        track_id_dict[track_id] = (
                            np.random.randint(0, 256), np.random.randint(0, 256), np.random.randint(0, 256))

                    frame = cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),
                                          track_id_dict[track_id], 2)
                    frame = cv2.putText(frame, str(track_id), (int(bbox[0]), int(bbox[1] + 70)),
                                        cv2.FONT_HERSHEY_SIMPLEX, 3, track_id_dict[track_id], 5)
                    frame = cv2.putText(frame, str(track_id), (int(bbox[0]), int(bbox[3] - 10)),
                                        cv2.FONT_HERSHEY_SIMPLEX, 3, track_id_dict[track_id], 5)

                # Add frame counter to frame corner
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.8
                font_color = (255, 255, 255)
                thickness = 1
                text = video_name + ' Frame: ' + str(frame_counter + 1) + '/' + str(total_frames)
                text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
                text_x = 10
                text_y = text_size[1] + 10
                cv2.putText(frame, text, (text_x, text_y), font, font_scale, font_color, thickness, cv2.LINE_AA)

                # Show the frame
                cv2.imshow('Video', frame)
            else:
                break

        # Delay between frames to slow down playback
        delay_time = 25  # milliseconds
        if slowdown == 0:
            if playing:
                key = cv2.waitKey(delay_time)
        elif slowdown == 1:
            if playing:
                key = cv2.waitKey(delay_time * 5)
        else:
            key = cv2.waitKey(0)

        if key == 27:  # If the "Esc" key is pressed, exit the loop
            break
        elif key == ord('r'):  # If the "r" key is pressed, restart the video
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            frame_counter = -1
            playing = True
        elif key==ord('b'):
            curr_frame=cap.get(cv2.CAP_PROP_POS_FRAMES)
            print(curr_frame)
            cap.set(cv2.CAP_PROP_POS_FRAMES,curr_frame-2)
            frame_counter = frame_counter -2
            playing = True
        elif key == ord(' ') and slowdown != 2:  # If the "space" bar  is pressed, stop/resume the video
            if not playing:
                frame_counter-=1
            playing = not playing

            # Pause or resume the video
        if not playing:
            cv2.waitKey(0)

        frame_counter += 1
    # Release the video capture object and close all windows
    cap.release()
    cv2.destroyAllWindows()

from json.decoder import JSONDecodeError


def find_all_IDs(path_to_folder_with_json):
    all_IDs = []
    for json_frame in os.listdir(path_to_folder_with_json):
        frame_path = os.path.join(path_to_folder_with_json, json_frame)
        data = []
        with open(frame_path, 'r') as f:
            try:
                data = json.load(f)
                for p in data:
                    if p['track_id'] not in all_IDs:
                        all_IDs.append(p['track_id'])
            except JSONDecodeError:
                pass
    return all_IDs


def assign_class_ID_range(path_to_folder_with_json, ID_to_assign, class_to_assign, interval):
    interval_ = []
    for i in range(int(len(interval) / 2)):
        interval_ += (list(range(interval[i * 2], interval[i * 2 + 1] + 1)))

    folder = os.listdir(path_to_folder_with_json)
    folder.sort()

    for idx_frame, json_frame in enumerate(folder):
        if idx_frame + 1 in interval_:
            frame_path = os.path.join(path_to_folder_with_json, json_frame)
            data = []
            with open(frame_path, 'r') as f:
                try:
                    data = json.load(f)
                    for p in data:
                        if p['track_id'] == ID_to_assign:
                            p['class'] = class_to_assign
                except JSONDecodeError:
                    pass
            if len(data) > 0:
                with open(frame_path, 'w') as f:
                    json.dump(data, f, indent=2)


def label_video(path_to_raw_video, path_to_folder_with_json, slowdown):
    total_frames = len(os.listdir(path_to_folder_with_json))
    json_files = [f for f in os.listdir(path_to_folder_with_json) if f.endswith('.json')]
    json_files.sort()
    
    # create new subfolder
    new_dir_name = os.path.basename(path_to_folder_with_json) + "_LabelIDs"
    new_dir_path = os.path.join(os.path.dirname(path_to_folder_with_json), new_dir_name)
    os.makedirs(new_dir_path, exist_ok=True)
    print(f'New folder with Labeled Track IDs is created successfully:', {new_dir_name})

    for file_name in json_files:
        file_path = os.path.join(path_to_folder_with_json, file_name)
        new_file_path = os.path.join(new_dir_path, file_name)
        copyfile(file_path, new_file_path)
        #print(json_files)

    # Assign to every ID class 0 (neutral)
    all_IDs = find_all_IDs(new_dir_path)
    for id in all_IDs:
        assign_class_ID_range(new_dir_path, id, 0, [1, total_frames])

    #visualize_video(path_to_raw_video, new_dir_path, slowdown)

    for id in all_IDs:

        # Ask user if ID is aggressor
        good_input = False
        while not good_input:
            is_class = input(f"Is ID {id} an aggressor?? Enter -1 to restart the video. Enter 0 for no, 1 for yes:")
            if is_class == "-1":
                visualize_video(path_to_raw_video, new_dir_path, slowdown)
            elif is_class not in ["0", "1"]:
                print(f"Invalid input.")
            elif is_class=="1":
                good_input = True

                good_interval_input=False
                while not good_interval_input:
                    interval_list = input(f"Write intervals of ID {id} being an aggressor (separated by commas). Enter -1 for restarting the video")
                    if interval_list == "-1":
                        visualize_video(path_to_raw_video, new_dir_path, slowdown)
                    else:
                        interval_list = [int(interv.strip()) for interv in interval_list.split(',')]
                        if len(interval_list)%2 == 0:
                            assign_class_ID_range(new_dir_path, id, 1, interval_list)
                            good_interval_input=True
                        else:   
                            print(f"Invalid input.")           
            else:
                good_input = True

        # Ask user if ID is victim
        good_input = False
        while not good_input:
            is_class = input(f"Is ID {id} a victim?? Enter -1 to restart the video. Enter 0 for no, 1 for yes:")
            if is_class == "-1":
                visualize_video(path_to_raw_video, new_dir_path, slowdown)
            elif is_class not in ["0", "1"]:
                print(f"Invalid input.")
            elif is_class=="1":
                good_input = True

                good_interval_input=False
                while not good_interval_input:
                    interval_list = input(f"Write intervals of ID {id} being a victim (separated by commas). Enter -1 for restarting the video")
                    if interval_list == "-1":
                        visualize_video(path_to_raw_video, new_dir_path, slowdown)
                    else:
                        interval_list = [int(interv.strip()) for interv in interval_list.split(',')]
                        if len(interval_list)%2 == 0:
                            assign_class_ID_range(new_dir_path, id, 2, interval_list)
                            good_interval_input=True
                        else:   
                            print(f"Invalid input.")           
            else:
                good_input = True
                

        visualize_video(path_to_raw_video, new_dir_path, slowdown)
  


def merge_id(path_to_raw_video, path_to_removedID_folder, slowdown):
    #visualize_video(path_to_raw_video, path_to_removedID_folder, slowdown)

    print("CURRENT VIDEO: " + path_to_raw_video)

    # All JSON files which need to be inspected based on user inputs
    json_files = os.listdir(path_to_removedID_folder)
    json_files.sort()

    merged_folder = path_to_removedID_folder + '_mergedID'
    # Create destination folder, if it is not already created
    isExist = os.path.exists(merged_folder)
    if not isExist:
        os.makedirs(merged_folder)

    # Ask user if merge is required for this video or not.
    merge_required = int(input("Do you want to merge any IDs? (1 = yes, anything else = no): "))

    # if no merging is required, then copy all corresponding JSON files to destination folder
    if (merge_required != 1):
        for file_name in json_files:
            src = os.path.join(path_to_removedID_folder, file_name)
            dst = os.path.join(merged_folder, file_name)
            shutil.copy(src, dst)

    else:
        # if merging is required, then do as under.
        for file_name in json_files:
            src = os.path.join(path_to_removedID_folder, file_name)
            dst = os.path.join(merged_folder, file_name)
            shutil.copy(src, dst)

        # variable used to continue looping
        continue_loop = 1
        while (continue_loop == 1):

            # Accept IDs to merge
            two_IDs = input("You should enter two IDs seperated by space to merge (for example <<1 2>> or <<4 5>>): ")
            two_IDs.strip()
            IDs = two_IDs.split(' ')
            IDs_to_merge = [int(ID) for ID in IDs]

            # Find out minimum ID value to keep after merging
            min_id_val = 99999
            for id in IDs_to_merge:
                if id < min_id_val:
                    min_id_val = id

            # For each JSON file
            for file_name in json_files:
                src = os.path.join(merged_folder, file_name)

                with open(src, 'r+') as f:
                    # Load contents of that JSON
                    data = json.load(f)  # list
                    data_to_write = data.copy()  # list

                    # Find out all IDs present in that JSON
                    track_ids = []
                    for track in range(len(data)):
                        track_ids.append(data[track]['track_id'])

                    # Create a list of IDs which are present in JSON and user has asked them to merge
                    tracks_present = [i for i in track_ids if i in IDs_to_merge]

                    track_to_be_deleted = []

                    # For each track in JSON
                    for track in range(len(data)):
                        if (len(tracks_present) == 2):
                            if (data[track]['track_id'] != min_id_val) and (data[track]['track_id'] in IDs_to_merge):
                                data_to_write.remove(data_to_write[track])
                        elif (len(tracks_present) == 1):
                            if (data[track]['track_id'] in IDs_to_merge):
                                data_to_write[track]['track_id'] = min_id_val

                    dst = os.path.join(merged_folder, file_name)
                    with open(dst, 'w') as fw:
                        fw.write(json.dumps(data_to_write, indent=4))

            again = input("Do you want to merge any other IDs? (1 = yes, anything else = no): ")
            if (again != '1'):
                continue_loop = 0

    visualize_video(path_to_raw_video, merged_folder, slowdown)

    label_video(path_to_raw_video, merged_folder, slowdown)
    
def remove_id(path_to_raw_video, path_to_folder_with_json, slowdown):
    # Call visualize_video function
    visualize_video(path_to_raw_video, path_to_folder_with_json, slowdown)

    name_of_folder_with_json = os.path.basename(path_to_folder_with_json)
    # Load all JSON files from the specified folder
    json_files = [f for f in os.listdir(path_to_folder_with_json) if f.endswith('.json')]

    # Ask user which IDs to remove
    remove_ids_str = input(
        f"Enter Track IDs to remove (comma separated) from {name_of_folder_with_json} (or enter -1 to keep all IDs): ")

    if remove_ids_str == "-1":
        print("No IDs will be removed")
        # Create new directory for updated JSON files
        new_dir_name = os.path.basename(path_to_folder_with_json) + "_RemovedIDs"
        new_dir_path = os.path.join(os.path.dirname(path_to_folder_with_json), new_dir_name)
        os.makedirs(new_dir_path, exist_ok=True)
        print(f'New folder with Removed Track IDs is created successfully:', {new_dir_name})

        # Iterate over JSON files and copy them to the new directory
        for file_name in json_files:
            file_path = os.path.join(path_to_folder_with_json, file_name)
            new_file_path = os.path.join(new_dir_path, file_name)
            shutil.copy(file_path, new_file_path)

    else:
        remove_ids = [int(id.strip()) for id in remove_ids_str.split(',')]
        # Create new directory for updated JSON files
        new_dir_name = os.path.basename(path_to_folder_with_json) + "_RemovedIDs"
        new_dir_path = os.path.join(os.path.dirname(path_to_folder_with_json), new_dir_name)
        os.makedirs(new_dir_path, exist_ok=True)
        print(f'New folder with Removed Track IDs is created successfully:', {new_dir_name})

        # Iterate over JSON files and update them
        for file_name in json_files:
            file_path = os.path.join(path_to_folder_with_json, file_name)
            new_file_path = os.path.join(new_dir_path, file_name)

            # Load JSON file
            with open(file_path, 'r') as f:
                data = json.load(f)

            # Create new list of dictionaries with IDs not in remove_ids
            new_data = [d for d in data if d['track_id'] not in remove_ids]

            # Save updated JSON file in the new_file_path which has the same name with _RemovedIDs
            with open(new_file_path, 'w') as f:
                json.dump(new_data, f, indent=4)

    # Call visualize_video function on the new directory
    # And visualize the video with new JSONs with removed ID
    visualize_video(path_to_raw_video, new_dir_path, slowdown)

    # call merge_id with new json files folder
    merge_id(path_to_raw_video, new_dir_path, slowdown)
    
# provide the main root folder that contains multiple folder
def process_subfolders(root_folder, slowdown=0):
    for subdir, dirs, files in os.walk(root_folder):
        json_files = [f for f in files if f.endswith('.json')]

        if json_files:
            json_folder_path = subdir

            folder_path = os.path.dirname(json_folder_path)

            subfolder_names = [n for n in os.listdir(folder_path) if
                               os.path.isdir(os.path.join(folder_path, n))]
            
            count1 = len(list(filter(lambda x: not x.startswith('.'), subfolder_names)))
            
            if (count1 == 2):
                video_folder_name = next(n for n in subfolder_names if
                                         n != os.path.basename(json_folder_path))
                video_folder_path = os.path.join(folder_path, video_folder_name)
                raw_video_path = os.path.join(video_folder_path, os.listdir(video_folder_path)[0])

                print(f'JSON files and MP4 found in', {folder_path}, 'now running remove_id')

                remove_id(raw_video_path, json_folder_path, slowdown = slowdown)

            else:
                break

        else:
            print(f"No JSON or MP4 files found in", {os.path.dirname(subdir)})


In [ ]:
process_subfolders('path/to/folder/with/raw/videos', slowdown=1 )